In [1]:
from time import sleep
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.chrome.options import Options

import threading
import time

In [2]:
chatbot = ChatBot('ANTON',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        {
            "import_path": "chatterbot.logic.BestMatch",
        },
        {
            'import_path': 'chatterbot.logic.MathematicalEvaluation',            
        },
        {
            'import_path': 'chatterbot.logic.UnitConversion',          
        }        
    ])

trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train("chatterbot.corpus.english")

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [3]:
driver=None

In [4]:
def whatsappWebConnection(chromeDriverPath):
    global driver

    chrome_options = Options() # Saving the last session
    chrome_options.add_argument("user-data-dir=selenium") 
    driver = webdriver.Chrome(chromeDriverPath, chrome_options=chrome_options)
    driver.get('https://web.whatsapp.com/')

    while True:
        time.sleep(1)
        try:
            appLoad = driver.find_element_by_class_name("m6ZEb") # The class only exists after the QR login page
            if appLoad:
                return
        except NoSuchElementException:
            pass

In [5]:
def getLastMessage(targetName):
        global driver
        target = driver.find_element_by_xpath('//span[@title= "{}"]'.format(targetName))
        target.click()

        messages = None

        while(1):
            try:
                messages = driver.find_elements_by_class_name("_3zb-j")
                newMessage = messages[-1].text
                return newMessage
            except (NoSuchElementException, StaleElementReferenceException) as e:
                pass

In [6]:
def sendMessage(targetName, msg):
        global driver
        target = driver.find_element_by_xpath('//span[@title= "{}"]'.format(targetName))
        target.click()

        msg_box = driver.find_element_by_xpath('//*[@id="main"]/footer/div[1]/div[2]/div/div[2]')
        msg_box.send_keys(msg)
        button = driver.find_element_by_class_name('_35EW6').click()

In [7]:
whatsappWebConnection('C:/users/tyagi/Downloads/chromedriver.exe')

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [8]:
def startAnton():
    threading.Timer(30.0, startAnton).start()
    msg=getLastMessage("ME") #name of your contact with whom you want to talk
    f = open("WhatsappMessages.txt",encoding="utf8")
    oldMessage=f.readline()
    if(msg!=oldMessage):
        response = chatbot.get_response(msg)
        print(msg,end=' : ')
        print(response)
        print()
        sendMessage("ME", str(response))#name of your contact with whom you want to talk
        f.write(str(response))
        f.close()
        time.sleep(5)

In [ ]:
startAnton()